## Problem Statement
Homonyms, or words that have multiple meanings depending on the context, to handle this problem we can use model that generate the embeddings of given word with context consideration such as BERT

In [1]:
%%writefile sentiment_analyzer.py
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.nn.functional import softmax

class SentimentAnalyzerManager:
    
    def __init__(self):
        model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = BertForSequenceClassification.from_pretrained(model_name)

    def predict_sentiment(self, sentence):
        inputs = self.tokenizer(sentence, return_tensors="pt", truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = self.model(**inputs)
        logits = outputs.logits
        probs = softmax(logits, dim=-1)
        prediction = torch.argmax(probs, dim=-1).item()
        sentiment = "negative" if prediction < 2 else "positive"
        return sentiment  

Writing sentiment_analyzer.py


In [6]:
import pandas as pd
from transformers import pipeline

import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.nn.functional import softmax


In [40]:
data = [{"Sentence" : "I hate the selfishness in you","label": "negative"},
{"Sentence": "I hate anyone who can hurt you","label": "positive"} ]

data = pd.DataFrame(data)
data.head()

,Sentence,label
0,I hate the selfishness in you,negative
1,I hate anyone who can hurt you,positive


In [41]:
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

In [46]:
def predict_sentiment(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probs = softmax(logits, dim=-1)
    prediction = torch.argmax(probs, dim=-1).item()
    sentiment = "negative" if prediction < 2 else "positive"
    return sentiment

In [47]:
data['predicted'] = data['Sentence'].apply(predict_sentiment)

In [48]:
data.head()

,Sentence,label,predicted
0,I hate the selfishness in you,negative,negative
1,I hate anyone who can hurt you,positive,positive
